In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

In [2]:
!git clone https://github.com/facebookresearch/ijepa.git

Cloning into 'ijepa'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 32 (delta 6), reused 3 (delta 3), pack-reused 5 (from 1)
Receiving objects: 100% (32/32), 33.02 KiB | 8.25 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [3]:
!wget https://dl.fbaipublicfiles.com/ijepa/IN1K-vit.h.14-300e.pth.tar

--2024-12-17 09:37:24--  https://dl.fbaipublicfiles.com/ijepa/IN1K-vit.h.14-300e.pth.tar
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.158.20.43, 108.158.20.111, 108.158.20.21, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.158.20.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10358004345 (9.6G) [application/x-tar]
Saving to: 'IN1K-vit.h.14-300e.pth.tar'

IN1K-vit.h.14-300e. 100%[===================>]   9.65G  26.2MB/s    in 7m 13s  

2024-12-17 09:44:38 (22.8 MB/s) - 'IN1K-vit.h.14-300e.pth.tar' saved [10358004345/10358004345]



In [4]:
import torch
import sys
from pathlib import Path
import yaml
from collections import OrderedDict
sys.path.append('/kaggle/working/ijepa/')
from src.models.vision_transformer import VisionTransformer
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from functools import partial
from torch.optim import AdamW
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from functools import partial
import copy
import time
from tqdm import tqdm
from torch.amp import autocast, GradScaler
from torch.optim.lr_scheduler import CosineAnnealingLR, LinearLR
from torch.optim import AdamW
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy.optimize import linear_sum_assignment
import numpy as np

In [5]:
# Load dataset
KITTI_ROOT = "/kaggle/input"
IMAGE_PATH = "/kaggle/input/data_object_image_2/training/image_2/"
LABEL_PATH = "/kaggle/input/data_object_label_2/training/label_2/"
CALIB_PATH = "/kaggle/input/data_object_calib/training/calib/"
CHECKPOINT_PATH = '/kaggle/working/IN1K-vit.h.14-300e.pth.tar'
model = VisionTransformer()

In [6]:
def load_pretrained_ijepa(checkpoint_path, model):
    """
    Load pretrained IJEPA weights into the model with proper key mapping
    """
    def verify_state_dict(state_dict):
        """Verify the state dict has expected structure"""
        # Check if we have encoder prefix
        has_encoder = any(k.startswith('encoder.') for k in state_dict.keys())
        
        required_keys = {
            'pos_embed', 'patch_embed.proj.weight', 'blocks.0.norm1.weight'
        }
        if has_encoder:
            required_keys = {f'encoder.{k}' for k in required_keys}
        
        missing_keys = required_keys - set(state_dict.keys())
        if missing_keys:
            print(f"Warning: Missing expected keys: {missing_keys}")
            return False
        return True

    try:
        # Load checkpoint with weights_only=True for security
        print(f"Loading checkpoint from {checkpoint_path}")
        checkpoint = torch.load(
            checkpoint_path,
            map_location='cuda',
            weights_only=True
        )
        
        # First, let's examine the checkpoint structure
        print("Checkpoint keys:", checkpoint.keys())
        
        # Handle different checkpoint formats
        if 'model' in checkpoint:
            state_dict = checkpoint['model']
        elif 'encoder' in checkpoint:
            state_dict = checkpoint['encoder']
        else:
            state_dict = checkpoint
        
        print("State dict keys sample:", list(state_dict.keys())[:5])
        
        # Create mapping for IJEPA keys to model keys
        key_mapping = {
            'encoder.pos_embed': 'pos_embed',
            'encoder.patch_embed': 'patch_embed',
            'encoder.blocks': 'blocks',
            'encoder.norm': 'norm',
            'pos_embed': 'pos_embed',
            'patch_embed': 'patch_embed',
            'blocks': 'blocks',
            'norm': 'norm'
        }
            
        # Remove 'module.' prefix and map keys
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            # Remove module prefix if present
            if k.startswith('module.'):
                k = k[7:]
            
            # Map keys from IJEPA format to model format
            mapped_key = k
            for old_prefix, new_prefix in key_mapping.items():
                if k.startswith(old_prefix):
                    mapped_key = k.replace(old_prefix, new_prefix)
                    break
            
            # Verify tensor shapes match
            if mapped_key in model.state_dict():
                if v.shape != model.state_dict()[mapped_key].shape:
                    print(f"Warning: Shape mismatch for {mapped_key}: checkpoint={v.shape}, model={model.state_dict()[mapped_key].shape}")
                    continue
                new_state_dict[mapped_key] = v
            else:
                print(f"Skipping key {mapped_key} as it's not in model state dict")
            
        # Load weights with error handling
        try:
            msg = model.load_state_dict(new_state_dict, strict=False)
            print(f"Successfully loaded pretrained weights")
            print(f"Missing keys: {msg.missing_keys}")
            print(f"Unexpected keys: {msg.unexpected_keys}")
            
            # Initialize missing keys
            for name, param in model.named_parameters():
                if name in msg.missing_keys:
                    if 'weight' in name:
                        if 'conv' in name or 'proj' in name:
                            nn.init.kaiming_normal_(param, mode='fan_out')
                        elif 'norm' in name:
                            nn.init.constant_(param, 1.0)
                        else:
                            nn.init.xavier_uniform_(param)
                    elif 'bias' in name:
                        nn.init.constant_(param, 0)
                        
        except Exception as e:
            print(f"Error loading state dict: {str(e)}")
            print("Keys in checkpoint:", new_state_dict.keys())
            print("Model state dict keys:", model.state_dict().keys())
            raise
            
        # Verify model parameters
        zero_params = sum(torch.all(p == 0) for p in model.parameters())
        if zero_params > 0:
            print(f"Warning: {zero_params} parameters are all zero")
            
        return model
        
    except Exception as e:
        print(f"Error loading checkpoint from {checkpoint_path}: {str(e)}")
        print("Initializing model with random weights")
        return model
    
def setup_ijepa_config(config_path=None):
    """
    Setup IJEPA configuration for object detection with correct dimensions
    """
    default_config = {
        'model': {
            'type': 'vit',
            'arch': 'huge',
            'patch_size': 14,
            'width': 1280,  
            'depth': 32,
            'num_heads': 16,
            'mlp_ratio': 4,
            'qkv_bias': True,
            'drop_path_rate': 0,
            'checkpoint_activations': False,
            'image_size': 224,
            'embed_dim': 1280,  
            'num_heads': 16,   
            'mlp_dim': 5120 
        },
        'data': {
            'type': 'kitti',
            'image_size': 224,
            'num_classes': 8  
        }
    }

    return default_config

def setup_ijepa_for_detection():
    """
    Setup IJEPA model for object detection with proper initialization
    """
    config = {
        'model': {
            'image_size': (224, 224),
            'patch_size': 14,
            'num_classes': 8,  
            'dim': 1280,       
            'depth': 32,       
            'heads': 16,       
            'mlp_dim': 5120,   
            'dropout': 0.0,
            'emb_dropout': 0.0,
            'embed_dim': 1280, 
            'num_heads': 16,  
            'mlp_ratio': 4,
            'qkv_bias': True,
            'in_chans': 3    
        }
    }

    # Initialize ViT with matching IJEPA architecture
    model = VisionTransformer(
        img_size=config['model']['image_size'],
        patch_size=config['model']['patch_size'],
        in_chans=config['model']['in_chans'],
        num_classes=config['model']['num_classes'],
        embed_dim=config['model']['embed_dim'],
        depth=config['model']['depth'],
        num_heads=config['model']['heads'],
        mlp_ratio=config['model']['mlp_ratio'],
        qkv_bias=config['model']['qkv_bias'],
        drop_rate=config['model']['dropout'],
        attn_drop_rate=config['model']['dropout'],
        drop_path_rate=0.0,
        norm_layer=partial(nn.LayerNorm, eps=1e-6),
        init_values=None,
        use_abs_pos_emb=True,
        init_std=0.02
    )

    return model, config

def main():
    # Paths
    pretrained_path = CHECKPOINT_PATH
    
    # Verify checkpoint exists
    if not os.path.exists(pretrained_path):
        raise FileNotFoundError(f"Checkpoint not found at {pretrained_path}")
        
    # Examine checkpoint structure
    print("Examining checkpoint structure...")
    checkpoint = torch.load(pretrained_path, map_location='cpu', weights_only=True)
    print("Checkpoint keys:", checkpoint.keys())
    
    # Setup model and config with error handling
    try:
        model, config = setup_ijepa_for_detection()
        print("Model state dict keys:", list(model.state_dict().keys())[:5])
    except Exception as e:
        print(f"Error setting up model: {str(e)}")
        return None, None
        
    # Load pretrained weights with validation
    try:
        model = load_pretrained_ijepa(pretrained_path, model)
    except Exception as e:
        print(f"Error loading pretrained weights: {str(e)}")
        
    # Move model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Print model statistics
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Model is on device: {next(model.parameters()).device}")
    
    return model, config

In [7]:
class DETRDetectionHead(nn.Module):
    def __init__(self, in_channels, num_classes, hidden_dim=256, num_queries=100):
        super().__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        
        # Transformer components with batch_first=True
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=8,
            dim_feedforward=2048,
            dropout=0.1,
            activation='relu',
            batch_first=True  # Set batch_first=True
        )
        
        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=6
        )
        
        # Learnable object queries
        self.query_embed = nn.Embedding(num_queries, hidden_dim)
        
        # Input projection
        self.input_proj = nn.Sequential(
            nn.LayerNorm(in_channels),
            nn.Linear(in_channels, hidden_dim),
            nn.GELU()
        )
        
        # Output FFNs
        self.class_embed = nn.Linear(hidden_dim, num_classes + 1)  # +1 for no-object
        self.bbox_embed = MLP(hidden_dim, hidden_dim, 4, 3)
        
    def forward(self, x):
        bs = x.shape[0]
        
        # Project input features
        x = self.input_proj(x)  # [batch_size, seq_len, hidden_dim]
        
        # Add positional queries
        queries = self.query_embed.weight.unsqueeze(0).repeat(bs, 1, 1)  # [batch_size, num_queries, hidden_dim]
        x = torch.cat([queries, x], dim=1)  # [batch_size, num_queries + seq_len, hidden_dim]
        
        # Transformer processing (now with batch_first=True)
        memory = self.transformer(x)  # [batch_size, num_queries + seq_len, hidden_dim]
        
        # Extract query outputs
        outputs_class = self.class_embed(memory[:, :self.num_queries])
        outputs_coord = self.bbox_embed(memory[:, :self.num_queries]).sigmoid()
        
        return outputs_class, outputs_coord

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super().__init__()
        self.num_layers = num_layers
        h = [hidden_dim] * (num_layers - 1)
        self.layers = nn.ModuleList(
            nn.Linear(n, k) for n, k in zip([input_dim] + h, h + [output_dim])
        )

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = F.relu(layer(x)) if i < self.num_layers - 1 else layer(x)
        return x

class IJEPADETRDetector(nn.Module):
    def __init__(self, backbone, num_classes, hidden_dim=256, num_queries=100):
        super().__init__()
        self.backbone = backbone
        self.detection_head = DETRDetectionHead(
            in_channels=backbone.embed_dim,
            num_classes=num_classes,
            hidden_dim=hidden_dim,
            num_queries=num_queries
        )
        
        # Freeze backbone initially
        for param in self.backbone.parameters():
            param.requires_grad = False
            
    def forward(self, x):
        features = self.backbone(x)
        return self.detection_head(features)

def create_detection_transforms(image_size=224):
    """Create transforms for detection"""
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

class DETRLoss(nn.Module):
    def __init__(self, num_classes, matcher_cost_class=1, matcher_cost_bbox=5, matcher_cost_giou=2):
        super().__init__()
        self.num_classes = num_classes
        self.matcher_cost_class = matcher_cost_class
        self.matcher_cost_bbox = matcher_cost_bbox
        self.matcher_cost_giou = matcher_cost_giou
        
        # Register buffers for class weights
        self.register_buffer('empty_weight', torch.ones(num_classes + 1))
        self.empty_weight[-1] = 0.1  # Lower weight for background class
        
    def calculate_giou(self, box1, box2):
        """Calculate GIoU between two boxes in (cx, cy, w, h) format"""
        # Convert to x1,y1,x2,y2
        x1_1, y1_1 = box1[..., 0] - box1[..., 2]/2, box1[..., 1] - box1[..., 3]/2
        x2_1, y2_1 = box1[..., 0] + box1[..., 2]/2, box1[..., 1] + box1[..., 3]/2
        x1_2, y1_2 = box2[..., 0] - box2[..., 2]/2, box2[..., 1] - box2[..., 3]/2
        x2_2, y2_2 = box2[..., 0] + box2[..., 2]/2, box2[..., 1] + box2[..., 3]/2
        
        # Intersection
        x1_i = torch.max(x1_1, x1_2)
        y1_i = torch.max(y1_1, y1_2)
        x2_i = torch.min(x2_1, x2_2)
        y2_i = torch.min(y2_1, y2_2)
        
        intersection = torch.clamp(x2_i - x1_i, min=0) * torch.clamp(y2_i - y1_i, min=0)
        
        # Union
        area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
        area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
        union = area1 + area2 - intersection
        
        # Enclosing box
        x1_e = torch.min(x1_1, x1_2)
        y1_e = torch.min(y1_1, y1_2)
        x2_e = torch.max(x2_1, x2_2)
        y2_e = torch.max(y2_1, y2_2)
        
        enclosing_area = (x2_e - x1_e) * (y2_e - y1_e)
        
        # GIoU
        iou = intersection / (union + 1e-6)
        giou = iou - (enclosing_area - union) / (enclosing_area + 1e-6)
        
        return giou

    def hungarian_matcher(self, outputs, targets):
        """Match predictions to targets using Hungarian algorithm"""
        bs, num_queries = outputs['pred_logits'].shape[:2]
        
        # Create cost matrix for each batch
        indices = []
        for b in range(bs):
            out_prob = outputs['pred_logits'][b].softmax(-1)
            out_bbox = outputs['pred_boxes'][b]
            
            # Handle empty targets
            if len(targets['labels'][b]) == 0:
                indices.append(([], []))
                continue
                
            tgt_ids = targets['labels'][b]
            tgt_bbox = targets['boxes'][b]
            
            # Classification cost
            cost_class = -out_prob[:, tgt_ids]
            
            # L1 cost
            cost_bbox = torch.cdist(out_bbox, tgt_bbox, p=1)
            
            # GIoU cost
            cost_giou = -self.calculate_giou(out_bbox.unsqueeze(1), tgt_bbox.unsqueeze(0))
            
            # Final cost matrix
            C = (self.matcher_cost_class * cost_class + 
                 self.matcher_cost_bbox * cost_bbox +
                 self.matcher_cost_giou * cost_giou)
            
            # Hungarian matching
            indices_b = linear_sum_assignment(C.cpu().detach().numpy())
            indices.append(indices_b)
            
        return [(torch.as_tensor(i, dtype=torch.int64), 
                 torch.as_tensor(j, dtype=torch.int64)) for i, j in indices]

    def forward(self, outputs, targets):
        """Forward pass"""
        # Restructure outputs and targets
        outputs = {
            'pred_logits': outputs[0],  # Classification predictions
            'pred_boxes': outputs[1]    # Box predictions
        }
        
        target_dict = {
            'labels': [t for t in targets[1]],  # labels_list
            'boxes': [b for b in targets[0]]    # boxes_list
        }
        
        # Match predictions to targets
        indices = self.hungarian_matcher(outputs, target_dict)
        
        # Compute losses
        num_boxes = sum(len(t) for t in target_dict['labels'])
        num_boxes = torch.as_tensor(num_boxes, dtype=torch.float, device=outputs['pred_logits'].device)
        
        # Classification loss
        src_logits = outputs['pred_logits']
        target_classes = torch.full(src_logits.shape[:2], self.num_classes,
                                  dtype=torch.int64, device=src_logits.device)
        
        for batch_idx, (pred_idx, tgt_idx) in enumerate(indices):
            if len(tgt_idx) > 0:  # Only update if we have targets
                target_classes[batch_idx, pred_idx] = target_dict['labels'][batch_idx][tgt_idx]
        
        loss_ce = F.cross_entropy(src_logits.transpose(1, 2), target_classes, self.empty_weight)
        
        # Box losses
        loss_bbox = torch.tensor(0., device=src_logits.device)
        loss_giou = torch.tensor(0., device=src_logits.device)
        
        for batch_idx, (pred_idx, tgt_idx) in enumerate(indices):
            if len(pred_idx) == 0:
                continue
                
            pred_boxes = outputs['pred_boxes'][batch_idx, pred_idx]
            target_boxes = target_dict['boxes'][batch_idx][tgt_idx]
            
            # L1 loss
            loss_bbox += F.l1_loss(pred_boxes, target_boxes, reduction='none').sum() / num_boxes
            
            # GIoU loss
            loss_giou += (1 - self.calculate_giou(pred_boxes, target_boxes)).sum() / num_boxes
        
        # Combine losses
        loss = loss_ce + 5 * loss_bbox + 2 * loss_giou
        
        loss_dict = {
            'loss': loss.item(),
            'loss_ce': loss_ce.item(),
            'loss_bbox': loss_bbox.item(),
            'loss_giou': loss_giou.item()
        }
        
        return loss, loss_dict

def train_step(model, batch, criterion, optimizer, device):
    """Modified training step"""
    images, boxes_list, labels_list = batch

    # Move images to device
    images = images.to(device)

    # Move boxes and labels to device
    boxes_list = [boxes.to(device) for boxes in boxes_list]
    labels_list = [labels.to(device) for labels in labels_list]

    # Forward pass
    cls_preds, bbox_preds = model(images)

    # Calculate loss
    loss, loss_dict = criterion(cls_preds, bbox_preds, labels_list, boxes_list)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss_dict

In [8]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
from pathlib import Path

class KITTIDataset(Dataset):
    """
    KITTI Dataset for Object Detection (Kaggle version)
    Directory structure:
    kitti_root/
        - data_object_image_2/
            - training/
            - testing/
        - data_object_label_2/
            - training/
        - data_object_calib/
            - training/
            - testing/
    """

    CLASSES = {
        'Car': 0,
        'Van': 1,
        'Truck': 2,
        'Pedestrian': 3,
        'Person_sitting': 4,
        'Cyclist': 5,
        'Tram': 6,
        'Misc': 7
    }

    def __init__(self, root_dir, split='train', transform=None):
        """
        Args:
            root_dir (str): Path to KITTI dataset root directory
            split (str): 'train' or 'test'
            transform: Optional transform to be applied on images
        """
        self.root_dir = Path(root_dir)
        self.split = split
        self.transform = transform

        # Set up paths based on new directory structure
        split_folder = 'training' if split == 'train' else 'testing'
        self.image_dir = self.root_dir / 'data_object_image_2' / split_folder / 'image_2'
        self.label_dir = self.root_dir / 'data_object_label_2' / 'training' / 'label_2' if split == 'train' else None

        # Get all image files
        self.image_files = sorted(self.image_dir.glob('*.png'))

        # Validate dataset structure
        self._validate_dataset()

    def _validate_dataset(self):
        """Validate the dataset structure"""
        assert self.image_dir.exists(), f"Image directory not found at {self.image_dir}"
        if self.split == 'train':
            assert self.label_dir.exists(), f"Label directory not found at {self.label_dir}"
            # Verify matching number of labels and images
            label_files = list(self.label_dir.glob('*.txt'))
            assert len(label_files) == len(self.image_files), \
                "Mismatch between number of images and labels"

    def _read_label_file(self, label_file):
        """
        Read KITTI label file and extract object information
        Returns:
            boxes: numpy array of bounding boxes [N, 4] (x1, y1, x2, y2)
            labels: numpy array of class labels [N]
        """
        boxes = []
        labels = []

        with open(label_file, 'r') as f:
            for line in f:
                parts = line.strip().split(' ')

                # Skip objects we don't want to detect
                class_name = parts[0]
                if class_name not in self.CLASSES:
                    continue

                # Extract 2D bounding box (x1, y1, x2, y2)
                bbox = [float(parts[4]), float(parts[5]),
                       float(parts[6]), float(parts[7])]

                boxes.append(bbox)
                labels.append(self.CLASSES[class_name])

        # Convert to numpy arrays
        boxes = np.array(boxes, dtype=np.float32)
        labels = np.array(labels, dtype=np.int64)

        return boxes, labels

    def _preprocess_box(self, box, image_size):
        """
        Convert absolute coordinates to normalized coordinates
        Args:
            box: numpy array [4] (x1, y1, x2, y2)
            image_size: tuple (width, height)
        Returns:
            normalized box coordinates
        """
        width, height = image_size

        # Clip coordinates to image boundaries
        x1 = np.clip(box[0], 0, width)
        y1 = np.clip(box[1], 0, height)
        x2 = np.clip(box[2], 0, width)
        y2 = np.clip(box[3], 0, height)

        # Normalize coordinates
        x1 = x1 / width
        y1 = y1 / height
        x2 = x2 / width
        y2 = y2 / height

        # Convert to center, width, height format
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        box_width = x2 - x1
        box_height = y2 - y1

        return np.array([center_x, center_y, box_width, box_height],
                       dtype=np.float32)

    def __len__(self):
        """Return the number of images in the dataset"""
        return len(self.image_files)

    def __getitem__(self, idx):
        """
        Get a sample from the dataset
        Returns:
            image: transformed image tensor
            boxes: tensor of bounding boxes [N, 4] (center_x, center_y, width, height)
            labels: tensor of class labels [N]
        """
        # Load image
        image_path = self.image_files[idx]
        image = Image.open(image_path).convert('RGB')
        original_size = image.size  # (width, height)

        # Load labels if in training mode
        if self.split == 'train':
            label_path = self.label_dir / (image_path.stem + '.txt')
            boxes, labels = self._read_label_file(label_path)

            # Convert boxes to normalized center format
            processed_boxes = np.array([
                self._preprocess_box(box, original_size) for box in boxes
            ])
        else:
            # For test set, return dummy targets
            processed_boxes = np.zeros((0, 4), dtype=np.float32)
            labels = np.zeros((0,), dtype=np.int64)

        # Apply transforms
        if self.transform:
            image = self.transform(image)

        # Convert to tensors
        boxes_tensor = torch.from_numpy(processed_boxes)
        labels_tensor = torch.from_numpy(labels)

        return image, boxes_tensor, labels_tensor

    def get_image_path(self, idx):
        """Get the path of an image given its index"""
        return str(self.image_files[idx])

def create_kitti_dataloader(
    root_dir,
    split='train',
    batch_size=32,
    num_workers=4,
    transform=None
):
    """
    Create a DataLoader for the KITTI dataset
    Args:
        root_dir (str): Path to KITTI dataset root
        split (str): 'train' or 'test'
        batch_size (int): Batch size
        num_workers (int): Number of worker processes
        transform: Optional transform to be applied on images
    Returns:
        DataLoader object
    """
    dataset = KITTIDataset(
        root_dir=root_dir,
        split=split,
        transform=transform
    )

    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=(split == 'train'),
        num_workers=num_workers,
        collate_fn=collate_fn,
        pin_memory=True
    )

def collate_fn(batch):
    """
    Custom collate function to handle variable number of objects per image
    Args:
        batch: list of tuples (image, boxes, labels)
    Returns:
        images: tensor of shape [batch_size, C, H, W]
        boxes: list of tensors of shape [N, 4]
        labels: list of tensors of shape [N]
    """
    images, boxes, labels = zip(*batch)

    # Stack images
    images = torch.stack(images)

    # Return as tuple
    return images, list(boxes), list(labels)

In [9]:
from torch.utils.data import SubsetRandomSampler
from sklearn.model_selection import train_test_split


def create_train_val_split(dataset, val_size=0.2, random_state=42):
    """
    Create train/validation split from the training dataset
    """
    all_indices = list(range(len(dataset)))
    train_indices, val_indices = train_test_split(
        all_indices, 
        test_size=val_size, 
        random_state=random_state
    )
    
    return train_indices, val_indices

def create_train_val_dataloaders(root_dir, batch_size=8, val_ratio=0.2, num_workers=2, transform=None):
    """Create separate train and validation dataloaders from training data"""
    # Create full training dataset
    if transform is None:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

    full_dataset = KITTIDataset(
        root_dir=root_dir,
        split='train',  # Use training data for both train and val
        transform=transform
    )

    # Create train/val splits
    train_indices, val_indices = create_train_val_split(full_dataset)

    # Create subset samplers
    train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

    # Create dataloaders
    train_loader = torch.utils.data.DataLoader(
        full_dataset,
        batch_size=batch_size,
        sampler=train_sampler,
        num_workers=num_workers,
        collate_fn=collate_fn,
        pin_memory=True
    )

    val_loader = torch.utils.data.DataLoader(
        full_dataset,
        batch_size=batch_size,
        sampler=val_sampler,
        num_workers=num_workers,
        collate_fn=collate_fn,
        pin_memory=True
    )

    return train_loader, val_loader


class EarlyStopping:
    """Early stopping utility to prevent overfitting"""
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# Add this validation function before the training function
def validate_model(model, val_loader, criterion, device):
    """Run validation loop"""
    model.eval()
    val_loss = AverageMeter()
    
    with torch.no_grad():
        for images, boxes_list, labels_list in val_loader:
            images = images.to(device)
            boxes_list = [boxes.to(device) for boxes in boxes_list]
            labels_list = [labels.to(device) for labels in labels_list]
            
            # Update autocast usage
            with autocast(device_type='cuda' if device=='cuda' else 'cpu'):
                cls_preds, bbox_preds = model(images)
                loss, _ = criterion(cls_preds, bbox_preds, labels_list, boxes_list)
            
            val_loss.update(loss.item(), images.size(0))
            
    return val_loss.avg

# Replace your existing train_ijepa_detector function with this improved version
def train_ijepa_detector(
    model,
    train_loader,
    val_loader,
    num_epochs=30,
    initial_lr=1e-4,
    device='cuda',
    unfreeze_backbone_epoch=5,
    max_objects=50,
    patience=7,
    gradient_clip_val=1.0,
    accumulation_steps=4
):
    model = model.to(device)
    criterion = DETRLoss(num_classes=8).to(device)  # 8 classes for KITTI
    optimizer = AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=initial_lr,
        weight_decay=0.01,
        betas=(0.9, 0.999)
    )

    # Cosine learning rate scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=num_epochs, eta_min=1e-6
    )

    best_val_loss = float('inf')
    best_model = None

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        # Progress bar
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')

        optimizer.zero_grad()
        for i, (images, boxes_list, labels_list) in enumerate(pbar):
            images = images.to(device)
            boxes_list = [boxes.to(device) for boxes in boxes_list]
            labels_list = [labels.to(device) for labels in labels_list]

            # Forward pass
            outputs = model(images)  # Returns (cls_preds, bbox_preds)
            
            # Format targets for DETR loss
            targets = (boxes_list, labels_list)
            
            # Calculate loss
            loss, loss_dict = criterion(outputs, targets)
            loss = loss / accumulation_steps

            # Backward pass
            loss.backward()

            # Gradient accumulation
            if (i + 1) % accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_val)
                optimizer.step()
                optimizer.zero_grad()

            running_loss += loss.item() * accumulation_steps

            # Update progress bar
            pbar.set_postfix({
                'loss': f'{running_loss/(i+1):.4f}',
                'cls_loss': f'{loss_dict["loss_ce"]:.4f}',
                'bbox_loss': f'{loss_dict["loss_bbox"]:.4f}',
                'giou_loss': f'{loss_dict["loss_giou"]:.4f}',
                'lr': f'{optimizer.param_groups[0]["lr"]:.6f}'
            })

            # Clear cache periodically
            if i % 10 == 0:
                torch.cuda.empty_cache()

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_steps = 0
        
        with torch.no_grad():
            for images, boxes_list, labels_list in val_loader:
                images = images.to(device)
                boxes_list = [boxes.to(device) for boxes in boxes_list]
                labels_list = [labels.to(device) for labels in labels_list]
                
                outputs = model(images)
                targets = (boxes_list, labels_list)
                loss, _ = criterion(outputs, targets)
                val_loss += loss.item()
                val_steps += 1
        
        val_loss = val_loss / val_steps
        scheduler.step()

        print(f'Epoch {epoch+1} - Train Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = copy.deepcopy(model.state_dict())

        # Memory cleanup
        torch.cuda.empty_cache()

    return best_model

# Update your SimpleDetectionEvaluator class with these improvements
class SimpleDetectionEvaluator:
    def __init__(self, model, device, confidence_threshold=0.3):
        self.model = model
        self.device = device
        self.confidence_threshold = confidence_threshold
        self.classes = {
            0: 'Car', 1: 'Van', 2: 'Truck', 3: 'Pedestrian',
            4: 'Person_sitting', 5: 'Cyclist', 6: 'Tram', 7: 'Misc'
        }
    
    @torch.no_grad()
    def predict_image(self, image):
        self.model.eval()
        image = image.to(self.device)
        if image.dim() == 3:
            image = image.unsqueeze(0)
            
        # Get predictions
        cls_preds, bbox_preds = self.model(image)
        
        # Process predictions
        scores = F.softmax(cls_preds[0], dim=-1)
        confidences, labels = scores.max(dim=-1)
        boxes = bbox_preds[0]
        
        # Filter by confidence
        mask = confidences > self.confidence_threshold
        filtered_boxes = boxes[mask]
        filtered_scores = confidences[mask]
        filtered_labels = labels[mask]
        
        return filtered_boxes.cpu(), filtered_scores.cpu(), filtered_labels.cpu()

# Add this new evaluation function
def evaluate_model(model, val_loader, device, num_samples=5):
    evaluator = SimpleDetectionEvaluator(model, device)
    model.eval()
    
    metrics = {
        'total_detections': 0,
        'class_detections': {i: 0 for i in range(8)},
        'avg_confidence': []
    }
    
    print("\nEvaluating model...")
    
    with torch.no_grad():
        for i, (images, target_boxes, target_labels) in enumerate(val_loader):
            if i >= num_samples:
                break
                
            image = images[0]
            boxes, scores, labels = evaluator.predict_image(image)
            
            # Update metrics
            metrics['total_detections'] += len(boxes)
            metrics['avg_confidence'].extend(scores.tolist())
            for label in labels:
                metrics['class_detections'][label.item()] += 1
    
    # Calculate average confidence
    metrics['avg_confidence'] = np.mean(metrics['avg_confidence']) if metrics['avg_confidence'] else 0
    
    # Print results
    print("\nEvaluation Results:")
    print(f"Total detections: {metrics['total_detections']}")
    print(f"Average confidence: {metrics['avg_confidence']:.4f}")
    print("\nDetections per class:")
    for class_id, count in metrics['class_detections'].items():
        print(f"{evaluator.classes[class_id]}: {count}")
    
    return metrics
    
# Update these values in create_train_val_dataloaders
batch_size = 8  # Reduced batch size
num_workers = 2  # Reduced workers

# Update the training transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

# Create dataloaders with proper batch size
train_loader, val_loader = create_train_val_dataloaders(
    root_dir=KITTI_ROOT,
    batch_size=8,  # Smaller batch size to avoid memory issues
    val_ratio=0.2,
    num_workers=2,
    transform=transform
)

In [10]:
MAX_OBJECTS = 50

In [11]:
# Set up model with correct parameters
backbone, config = setup_ijepa_for_detection()
backbone = load_pretrained_ijepa(CHECKPOINT_PATH, backbone)

# Create detector
model = IJEPADETRDetector(
    backbone=backbone,
    num_classes=len(KITTIDataset.CLASSES),
    num_queries=MAX_OBJECTS,
    hidden_dim=256
)

# Move to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

Loading checkpoint from /kaggle/working/IN1K-vit.h.14-300e.pth.tar
Checkpoint keys: dict_keys(['encoder', 'predictor', 'opt', 'scaler', 'target_encoder', 'epoch', 'loss', 'batch_size', 'world_size', 'lr'])
State dict keys sample: ['module.pos_embed', 'module.patch_embed.proj.weight', 'module.patch_embed.proj.bias', 'module.blocks.0.norm1.weight', 'module.blocks.0.norm1.bias']
Successfully loaded pretrained weights
Missing keys: []
Unexpected keys: []


In [12]:
# Modify training parameters
trained_model = train_ijepa_detector(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=40,  
    initial_lr=1e-4,  
    device=device,
    unfreeze_backbone_epoch=5,
    max_objects=MAX_OBJECTS,
    patience=15,  
    gradient_clip_val=0.1  
)

Epoch 1/40: 100%|██████████| 748/748 [06:23<00:00,  1.95it/s, loss=3.5225, cls_loss=1.0080, bbox_loss=0.1083, giou_loss=0.6314, lr=0.000100]


Epoch 1 - Train Loss: 3.5225, Val Loss: 2.9167


Epoch 2/40: 100%|██████████| 748/748 [06:21<00:00,  1.96it/s, loss=2.9334, cls_loss=1.0688, bbox_loss=0.0863, giou_loss=0.5687, lr=0.000100]


Epoch 2 - Train Loss: 2.9334, Val Loss: 2.8942


Epoch 3/40: 100%|██████████| 748/748 [06:21<00:00,  1.96it/s, loss=2.8237, cls_loss=1.0130, bbox_loss=0.1005, giou_loss=0.5669, lr=0.000099]


...

Epoch 38 - Train Loss: 1.6558, Val Loss: 1.6516


Epoch 39/40: 100%|██████████| 748/748 [06:21<00:00,  1.96it/s, loss=1.6522, cls_loss=0.5672, bbox_loss=0.0554, giou_loss=0.4449, lr=0.000002]


Epoch 39 - Train Loss: 1.6522, Val Loss: 1.6549


Epoch 40/40: 100%|██████████| 748/748 [06:21<00:00,  1.96it/s, loss=1.6530, cls_loss=0.5487, bbox_loss=0.0566, giou_loss=0.3498, lr=0.000001]


Epoch 40 - Train Loss: 1.6530, Val Loss: 1.6628


In [13]:
def calculate_iou(box1, box2):
    """
    Calculate IOU between box1 and box2
    Boxes are in format: (center_x, center_y, width, height)
    """
    # Convert to x1, y1, x2, y2 format
    box1_x1 = box1[0] - box1[2]/2
    box1_y1 = box1[1] - box1[3]/2
    box1_x2 = box1[0] + box1[2]/2
    box1_y2 = box1[1] + box1[3]/2
    
    box2_x1 = box2[0] - box2[2]/2
    box2_y1 = box2[1] - box2[3]/2
    box2_x2 = box2[0] + box2[2]/2
    box2_y2 = box2[1] + box2[3]/2
    
    # Calculate intersection
    x1 = max(box1_x1, box2_x1)
    y1 = max(box1_y1, box2_y1)
    x2 = min(box1_x2, box2_x2)
    y2 = min(box1_y2, box2_y2)
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    
    # Calculate union
    box1_area = box1[2] * box1[3]
    box2_area = box2[2] * box2[3]
    union = box1_area + box2_area - intersection
    
    return intersection / (union + 1e-6)
    
def evaluate_detection(model, test_loader, device, iou_threshold=0.5):
    model.eval()
    total_ious = []
    class_ious = {i: [] for i in range(8)}
    class_metrics = {i: {'correct': 0, 'total': 0, 'ious': []} for i in range(8)}
    
    with torch.no_grad():
        for images, target_boxes, target_labels in tqdm(test_loader, desc="Evaluating"):
            images = images.to(device)
            cls_preds, bbox_preds = model(images)
            
            for i in range(len(images)):
                scores = F.softmax(cls_preds[i], dim=-1)
                confidences, pred_labels = scores.max(dim=-1)
                pred_boxes = bbox_preds[i]
                
                mask = confidences > 0.3
                filtered_boxes = pred_boxes[mask]
                filtered_labels = pred_labels[mask]
                
                if len(target_boxes[i]) > 0 and len(filtered_boxes) > 0:
                    for gt_box, gt_label in zip(target_boxes[i], target_labels[i]):
                        max_iou = 0
                        best_pred_label = None
                        
                        for pred_box, pred_label in zip(filtered_boxes, filtered_labels):
                            iou = calculate_iou(gt_box.cpu().numpy(), pred_box.cpu().numpy())
                            if iou > max_iou:
                                max_iou = iou
                                best_pred_label = pred_label.item()
                        
                        gt_label = gt_label.item()
                        if max_iou >= iou_threshold:
                            total_ious.append(max_iou)
                            class_ious[gt_label].append(max_iou)
                            class_metrics[gt_label]['total'] += 1
                            if best_pred_label == gt_label:
                                class_metrics[gt_label]['correct'] += 1

    # Calculate metrics
    mean_iou = np.mean(total_ious) if total_ious else 0
    class_mean_ious = {k: np.mean(v) if v else 0 for k, v in class_ious.items()}
    class_accuracies = {k: v['correct']/v['total'] if v['total'] > 0 else 0 
                       for k, v in class_metrics.items()}

    # Print results
    print("\nDetection Results:")
    print(f"Overall Mean IOU: {mean_iou:.4f}")
    print(f"Total detections: {len(total_ious)}")
    print("\nPer-class Results:")
    for class_id in class_metrics:
        class_name = IDX_TO_CLASS[class_id]
        accuracy = class_accuracies[class_id]
        iou = class_mean_ious[class_id]
        total = class_metrics[class_id]['total']
        print(f"{class_name}:")
        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  IoU: {iou:.4f}")
        print(f"  Total detections: {total}")

    return {
        'mean_iou': mean_iou,
        'class_ious': class_mean_ious,
        'class_accuracies': class_accuracies,
        'total_detections': len(total_ious)
    }
IDX_TO_CLASS = {v: k for k, v in KITTIDataset.CLASSES.items()}

# Use the function
results = evaluate_detection(model, val_loader, device)

# Now you can evaluate
print(f"\nOverall Results:")
print(f"Mean IoU: {results['mean_iou']:.4f}")
print(f"Total detections: {results['total_detections']}")

print("\nResults by Class:")
for class_id, acc in results['class_accuracies'].items():
    class_name = IDX_TO_CLASS[class_id]
    iou = results['class_ious'][class_id]
    print(f"{class_name}:")
    print(f"  Accuracy: {acc:.4f}")
    print(f"  IoU: {iou:.4f}")

Evaluating: 100%|██████████| 188/188 [01:43<00:00,  1.81it/s]


Detection Results:
Overall Mean IOU: 0.7104
Total detections: 6615

Per-class Results:
Car:
  Accuracy: 0.8694
  IoU: 0.7201
  Total detections: 5131
Van:
  Accuracy: 0.4491
  IoU: 0.6900
  Total detections: 452
Truck:
  Accuracy: 0.6579
  IoU: 0.7199
  Total detections: 190
Pedestrian:
  Accuracy: 0.6921
  IoU: 0.6581
  Total detections: 484
Person_sitting:
  Accuracy: 0.5652
  IoU: 0.6554
  Total detections: 23
Cyclist:
  Accuracy: 0.4726
  IoU: 0.6451
  Total detections: 146
Tram:
  Accuracy: 0.8052
  IoU: 0.7011
  Total detections: 77
Misc:
  Accuracy: 0.4554
  IoU: 0.6592
  Total detections: 112

Overall Results:
Mean IoU: 0.7104
Total detections: 6615

Results by Class:
Car:
  Accuracy: 0.8694
  IoU: 0.7201
Van:
  Accuracy: 0.4491
  IoU: 0.6900
Truck:
  Accuracy: 0.6579
  IoU: 0.7199
Pedestrian:
  Accuracy: 0.6921
  IoU: 0.6581
Person_sitting:
  Accuracy: 0.5652
  IoU: 0.6554
Cyclist:
  Accuracy: 0.4726
  IoU: 0.6451
Tram:
  Accuracy: 0.8052
  IoU: 0.7011
Misc:
  Accuracy: 0.455